In [42]:
def dump_ground_program(solver):
    print(solver.prg)

In [43]:
from incraf import incr_af_solver

from clingo.core import TruthValue
from clingo.control import Control
from clingo.symbol import Number, Function, Symbol, String
from clingo.solving import Model, SolveResult

from clingox.program import Program, ProgramObserver
from clingox.backend import SymbolicBackend

In [44]:
def on_model(m: Model):
    print(f"Model {m.number}:", m)

# Static

In [45]:
filename = "asp/af1.lp"
semantic = "stable"

In [46]:
ctl = Control(arguments=["--models=0"])
prg = Program()
ctl.register_observer(ProgramObserver(prg))

ctl.load(filename)
ctl.load(f"dung/{semantic}.dl")
ctl.load("asp/filter.lp")

ctl.ground()

In [47]:
ctl.solve(on_model=on_model).satisfiable

Model 1: in(a) in(c)


True

# Incremental

In [48]:
af = incr_af_solver(semantic, filename)

In [49]:
af.solve_enum()

DEBUG Model 1: in(a) in(c)


[[Function('in', [Function('a', [], True)], True),
  Function('in', [Function('c', [], True)], True)]]

In [51]:
af.add_argument(Function("arg", [Function('d', [])]))
af.del_attack(attack=Function("att", [Function('b', []), Function('a', [])]))
af.del_attack(attack=Function("att", [Function('c', []), Function('b', [])]))
af.add_attack(attack=Function("att", [Function('c', []), Function('d', [])]))
af.add_attack(attack=Function("att", [Function('d', []), Function('a', [])]))
af.add_attack(attack=Function("att", [Function('b', []), Function('c', [])]))

RuntimeError: redefinition of atom <'defeated(a)',4>

In [52]:
dump_ground_program(af)

 :- out(a), not defeated(a).
 :- out(b), not defeated(b).
 :- out(c), not defeated(c).
 :- att(a,b), in(b), in(a).
 :- att(b,a), in(a), in(b).
 :- att(c,b), in(b), in(c).
 :- __x17, not __x19.
 :- __x20, in(d), in(c).
out(a) :- arg(a), not in(a).
in(a) :- arg(a), not out(a).
defeated(a) :- att(b,a), in(b).
defeated(a) :- __x21, in(d).
out(b) :- arg(b), not in(b).
in(b) :- arg(b), not out(b).
defeated(b) :- att(a,b), in(a).
defeated(b) :- att(c,b), in(c).
out(c) :- arg(c), not in(c).
in(c) :- arg(c), not out(c).
__x17 :- __x16, not in(d).
in(d) :- __x16, not __x17.
__x19 :- __x20, in(c).
#external arg(a). [True]
#external defeated(a). [False]
#external arg(b). [True]
#external defeated(b). [False]
#external arg(c). [True]
#external defeated(c). [False]
#external att(a,b). [True]
#external att(b,a). [True]
#external att(b,a). [False]
#external att(c,b). [True]
#external att(c,b). [False]
#external __x16. [True]
#external __x19. [False]
#external __x20. [True]
#external __x21. [True]
